# Read Data

In [1]:
#!pip install plotly_express
#!pip install pandas 
#!pip instal imblearn
#!pip install xgboost
#!pip install lightgbm

In [110]:
# import libraries
import pandas as pd 
import plotly_express as px 
import json
import joblib
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import lightgbm


In [3]:
# first dataset
try:
    url1 = 'datasets/tinder.csv'
    tinder = pd.read_csv(url1, on_bad_lines='skip')
    print('local')
except:
    print('remote')
    url1 = 'https://raw.githubusercontent.com/dataPracticum/code_pudding_v2/main/Tinder.csv'
    tinder = pd.read_csv(url1, on_bad_lines='skip', names=['id', 'usage', 'university', 'answer', 'matches', 'percentage', 'relationship'])

local


In [4]:
tinder.head()

,id,usage,university,answer,matches,percentage,relationship
0,292890.8970,web,"Meridian, Idaho",No,0.000000,0.000000,0
1,292887.9870,web,"Meridian, Idaho",No,0.000000,0.000000,0
2,292894.0656,gender,"Meridian, Idaho",No,499.173606,0.225255,0
3,292887.1180,web,"Meridian, Idaho",No,0.000000,0.000000,0
4,292893.6561,gender,"Meridian, Idaho",No,455.925963,0.211360,0


In [5]:
# function to convert to json
def convert_json(df):
    df_json = df.to_json()
    data = json.loads(df_json)
    new_df = json.dumps(data, indent=2)
    return print(new_df)

In [6]:
# convert_json(tinder)

# Introduction

##### Data

This mini dataset consists of questions answered by millennials from various universities based on their usage of Tinder app

##### Background

Tinder is a popular dating app that allows users to create a profile and swipe through potential matches based on their location and personal preferences. Users can indicate their interest in a match by swiping right, or they can swipe left to pass. If two users both swipe right on each other, they are considered a match and can communicate with each other through the app's messaging system. Tinder's algorithm also takes into account factors such as mutual friends and common interests to suggest potential matches. The app is known for its simple and user-friendly interface, and it is available on both iOS and Android devices.

Tinder users are typically individuals who are looking for romantic or sexual connections with others. The app is used by a wide range of people, including singles of various ages, sexual orientations, and backgrounds. While the app has been criticized in the past for promoting superficiality and "hookup culture," many users also report finding meaningful relationships through the platform. Tinder has a global user base, and the app is available in multiple languages. The demographics of Tinder users vary by location, but in general, the app tends to be popular among young adults aged 18 to 34.

Men use dating apps like Tinder more frequently than women, with 75.8% of the US Tinder user base being male, according to Statista as of March 2021. Men tend to like a large proportion of the women they view but receive only a small fraction of matches in return. Women are more selective about swiping, match at a higher rate than men, and write longer messages. 

##### Tinder U

Tinder has seen significant success on college campuses, particularly in the United States, with a large proportion of its user base consisting of college students. The app's emphasis on swiping through potential matches based on appearance and proximity, as well as its ease of use and social integration, have made it popular among college students looking for casual relationships, hookups, or friendships. Tinder has also partnered with a number of universities to promote the app on campus and to provide special features and events for students, further cementing its popularity among this demographic. However, while Tinder has been successful at connecting college students, some studies suggest that its emphasis on physical appearance and casual dating may contribute to a culture of hookups and promiscuity on college campuses.

##### Criticism

Apps like Tinder can give people the impression that there are an abundance of potential mates out there, leading to short-term dating and less satisfaction with partner choices. Prior to Tinder, most online dating services matched people based on autobiographical information, but Tinder places a higher weight on first impressions. 

On Tinder, the emphasis is often on casual dating and hookups, which may make it more difficult to find a long-term relationship. While there are certainly people who have met their partners on Tinder and formed meaningful relationships, the app's design and culture tend to prioritize superficial attraction and quick matches over getting to know someone on a deeper level.

Non-Tinder methods of meeting people, such as through mutual friends, social events, or online dating sites that focus on long-term relationships, may offer a better chance of finding a compatible partner. These methods often provide more opportunities for getting to know someone's personality, values, and interests before deciding to pursue a relationship.

In a study of Tinder users in New York City and London, men liked a large proportion of profiles but received returning likes only 0.6% of the time, while women were much more selective but received matches 10% of the time. By sending out questionnaires to frequent Tinder users, researchers discovered that men tend to like a large proportion of profiles they see to increase their chances of getting a match, leading to a feedback loop.

##### Comparisons

Compared to other dating apps, Tinder has a larger user base and is more widely known. Other dating apps, such as Bumble and Hinge, have different features and target different audiences. Bumble requires women to make the first move and is marketed towards more serious relationships, while Hinge emphasizes users' interests and connections through mutual friends.

## Tinder

In [7]:
# look at data
tinder.head()

,id,usage,university,answer,matches,percentage,relationship
0,292890.8970,web,"Meridian, Idaho",No,0.000000,0.000000,0
1,292887.9870,web,"Meridian, Idaho",No,0.000000,0.000000,0
2,292894.0656,gender,"Meridian, Idaho",No,499.173606,0.225255,0
3,292887.1180,web,"Meridian, Idaho",No,0.000000,0.000000,0
4,292893.6561,gender,"Meridian, Idaho",No,455.925963,0.211360,0


In [8]:
# missing values
tinder.isna().sum()

id              0
usage           0
university      0
answer          0
matches         0
percentage      0
relationship    0
dtype: int64

In [9]:
# looking for duplicates
tinder[tinder.duplicated()]

,id,usage,university,answer,matches,percentage,relationship
1295,292889.9062,web,"Meridian, Idaho",No,0.0,0.0,0
1636,292887.7623,web,"Meridian, Idaho",No,0.0,0.0,0


Different id's so they are not actually duplicates

In [10]:
# values of answer
tinder.answer.value_counts()

No                    1571
Yes                    164
I don't use Tinder     161
Name: answer, dtype: int64

In [11]:
# tinder columns
tinder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1896 entries, 0 to 1895
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1896 non-null   float64
 1   usage         1896 non-null   object 
 2   university    1896 non-null   object 
 3   answer        1896 non-null   object 
 4   matches       1896 non-null   float64
 5   percentage    1896 non-null   float64
 6   relationship  1896 non-null   int64  
dtypes: float64(3), int64(1), object(3)
memory usage: 103.8+ KB


Tinder is a relatively small dataset with 1,896 values. Target value is relationship, where it determines whether the individual found a relationship using the app. We can use this data to run a ML model to predict wether an individual will get a relationship from Tinder. 

Usage indicates the medium of usage, and Tinder has a mobile-based, web-based app as well as a separate feature called Tinder U. Not sure what Gender represents.

### EDA

In [12]:
# correlation
tinder.corr()

,id,matches,percentage,relationship
id,1.000000,-0.429790,0.164302,0.167766
matches,-0.429790,1.000000,0.412200,0.186139
percentage,0.164302,0.412200,1.000000,0.311429
relationship,0.167766,0.186139,0.311429,1.000000


In [13]:
# summary statistics
tinder.describe()

,id,matches,percentage,relationship
count,1896.000000,1896.000000,1896.000000,1896.000000
mean,292999.394750,446.881267,0.254098,0.350738
std,227.809806,494.110578,0.220809,0.477327
min,292881.000000,0.000000,0.000000,0.000000
25%,292888.196925,0.000000,0.000000,0.000000
50%,292893.904400,400.439633,0.257873,0.000000
75%,292902.568750,845.560798,0.396787,1.000000
max,293785.000000,1969.000000,1.000000,1.000000


In [14]:
# Skew
tinder.skew()

C:\Users\XIX\AppData\Local\Temp\ipykernel_1864\3200751176.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  tinder.skew()


id              2.044382
matches         0.716367
percentage      0.816940
relationship    0.626066
dtype: float64

In [15]:
# answers to question: do you use tinder?
tinder.answer.value_counts()

No                    1571
Yes                    164
I don't use Tinder     161
Name: answer, dtype: int64

In [16]:
# compile no answers
tinder.answer.replace('I don\'t use Tinder', 'No', inplace=True)

In [17]:
# answers to question: do you use tinder?
tinder.answer.value_counts()

No     1732
Yes     164
Name: answer, dtype: int64

In [18]:
# use tinder and found relationship
tinder[tinder.answer=='Yes'].relationship.value_counts()

1    86
0    78
Name: relationship, dtype: int64

In [19]:
# relationship found using tinder 
print(86/(86+78) * 100, '%')

52.4390243902439 %


In [20]:
# dont use tinder
tinder[tinder.answer=='No'].relationship.value_counts()

0    1153
1     579
Name: relationship, dtype: int64

In [21]:
# relationship found not using tinder
print(579/(1153+579) * 100, '%')

33.429561200923786 %


In [22]:
# dataframe comparing relationship success
data = {'category': ['Tinder', 'Not Tinder'], 'percentage':[52.44, 33.43]}
comparison = pd.DataFrame(data)

In [23]:
# convert_json(comparison)

##### Fig 1

In [24]:
fig1 = px.bar(comparison, x='category', y='percentage', color='category', title='Relationship Success Rates')
fig1

##### Fig 1 Description

According to the data, Tinder has a higher success rate, compared to non tinder users. Tinder users find a relationship in 52% of cases, while those that do not use tinder find a relationship in only 33% of cases. 

In [25]:
# fig1.write_html('HTML/fig1.html')

In [26]:
# convert_json(px.bar(comparison, x='category', y='percentage', color='category', title='Relationship Success Rates'))

In [27]:
# university names
tinder.university.value_counts()

Meridian, Idaho                       1421
Westport, CT                            14
University of Pennsylvania              12
University of Mississippi               10
University of Washington                10
                                      ... 
Boston College                           1
Southern Methodist University            1
Mississippi State University             1
Washington University in St. Louis       1
North Carolina State                     1
Name: university, Length: 148, dtype: int64

In [28]:
# medium of usage
tinder.usage.value_counts()

gender        488
mobile        483
university    464
web           461
Name: usage, dtype: int64

In [29]:
# target values
tinder.relationship.value_counts()

0    1231
1     665
Name: relationship, dtype: int64

##### Fig 2

In [30]:
# convert_json(tinder.answer.value_counts())

In [31]:
fig2 = px.pie(tinder.answer, names='answer', title='Total Percentage of Tinder Users', template='plotly_white', height=600)
fig2

##### Fig 2 Description

Roughly 91% of the data represents individuals that do not use tinder, while the remaining individuals use Tinder. This illustrates that the data is skewed towards individuals that do not use tinder. 

In [32]:
# fig2.write_html('HTML/fig2.html')

In [33]:
# convert_json(px.pie(tinder.answer, names='answer', title='Total Percentage of Tinder Users', template='plotly_white', height=600))

More people do not use Tinder

##### Fig 3

In [34]:
# convert_json(tinder.relationship.value_counts())

In [35]:
fig3 = px.pie(tinder.relationship, names='relationship', title='Total Percentage of Relationships', template='plotly_white', height=600)
fig3

##### Fig 3 Description

Within the dataset, 64.9% of respondents are in a relationship, while the remaining 35.1% are not in a relationship. This shows a class imbalance within the dataset. 

In [36]:
# fig3.write_html('HTML/fig3.html')

In [37]:
# convert_json(px.pie(tinder.relationship, names='relationship', title='Total Percentage of Relationships', template='plotly_white', height=600))

More people are not ina relationship

##### Fig 4

In [38]:
# convert_json(tinder[tinder.answer=='Yes'].relationship.value_counts())

In [39]:
fig4 = px.pie(tinder[tinder.answer=='Yes'].relationship, names='relationship', title='Relationships Found Using Tinder', template='plotly_white', height=600)
fig4

In [40]:
# fig4.write_html('HTML/fig4.html')

In [41]:
# convert_json(px.pie(tinder[tinder.answer=='Yes'].relationship, names='relationship', title='Relationships Found Using Tinder', template='plotly_white', height=600))

A little more than half of the people that use tinder have found a relationship using the app

##### Fig 5

In [42]:
# convert_json(tinder[tinder.answer=='No'].relationship.value_counts())

In [43]:
fig5 = px.pie(tinder[tinder.answer=='No'].relationship, names='relationship', title='Found a Relationship Without Tinder', template='plotly_white', height=600,)
fig5

In [44]:
# fig5.write_html('HTML/fig5.html')

In [45]:
# convert_json(px.pie(tinder[tinder.answer=='No'].relationship, names='relationship', title='Found a Relationship Without Tinder', template='plotly_white', height=600,))

A third of People not using Tinder have found a relationship

##### FIG 6, 7, 8, 9, 10

In [46]:
columns = ['usage', 'answer', 'matches', 'percentage',
       'relationship']
for column in columns:
    px.histogram(tinder[column], title='Distribution of '+ str.upper(column), labels={'value': str.upper(column)}, template='plotly_white', color_discrete_sequence=['red']).show()

In [47]:
# columns = ['usage', 'answer', 'matches', 'percentage',
#       'relationship']
# for column in columns:
#    convert_json(px.histogram(tinder[column], title='Distribution of '+ str.upper(column), labels={'value': str.upper(column)}, template='plotly_white', color_discrete_sequence=['red'])) 

In [48]:
# fig10 = px.histogram(tinder.relationship, title='Distribution of Relationship', labels={'value': 'Relationship'}, template='plotly_white', color_discrete_sequence=['red'])
# fig10

In [49]:
#  fig10.write_html('HTML/fig10.html')

In [50]:
# number of unique universities
tinder.university.nunique()

148

There are 148 different universities in the dataset

In [51]:
# The top 20 different universities
tinder.university.value_counts().nlargest(20)

Meridian, Idaho                         1421
Westport, CT                              14
University of Pennsylvania                12
University of Mississippi                 10
University of Washington                  10
University of Southern California          9
Washington State University                8
University of Tampa                        8
Wesleyan University                        8
University of Wisconsin, Eau Claire        8
University of California, Santa Cruz       8
University of California, Berkeley         8
Williams College                           7
Vassar College                             7
Vanderbilt University                      7
University of Buffalo                      7
University of Oregon                       7
Union College                              7
Whatsgoodly University                     6
University of Delaware                     6
Name: university, dtype: int64

##### Fig 11

In [52]:
fig11 = px.bar(tinder.university.value_counts().nlargest(20), title='Distribution of Most Common Universities', labels={'value': 'Universities'}, template='plotly_white', color_discrete_sequence=['red'])
fig11

In [53]:
# anomalies in data
tinder.query("university=='Male respondents'")

,id,usage,university,answer,matches,percentage,relationship
201,292897.0,gender,Male respondents,No,574.0,0.259,0
1014,292895.0,gender,Male respondents,No,1172.0,0.528,0
1035,292893.0,gender,Male respondents,Yes,472.0,0.213,1


In [54]:
# anomalies in data
tinder.query("university=='Female respondents'")

,id,usage,university,answer,matches,percentage,relationship
515,292903.0,gender,Female respondents,No,516.0,0.315,0
961,292901.0,gender,Female respondents,No,797.0,0.487,1
1277,292899.0,gender,Female respondents,Yes,325.0,0.198,0


In [55]:
# fig11.write_html('HTML/fig11.html')

Most common university is Meridian in Ohio. The other universities make up a small percentage, roughly 15%  of the rest of the schools.

##### Fig 12, 13

In [56]:
columns = ['matches', 'percentage']
for column in columns:
    px.box(tinder[column], title='Distribution of '+ str.upper(column), labels={'value': str.upper(column)}, template='plotly_white', color_discrete_sequence=['red']).show()

In [57]:
# fig13.write_html('HTML/fig13.html')

Average matches are around 400, with the max at 1969. The minimm amount of matches is 0, while the upper quartile is 846. Most people get matches on .25% of swipes, while the interquartile range is 0-0.4%.  

##### Fig 14

In [58]:
fig14 = px.imshow(tinder.corr(), text_auto=True, title='Tinder Correlations', template='plotly_white', aspect='equal', height=600, color_continuous_scale='hot')
fig14

The greatest correlation with the target of a a relationship is match percentage. The second highest correlation is with matches. Since percentage represents the match percentage, it is intuitive that more mutual matches would be correlated with being in a relationship. 

In [59]:
# fig14.write_html('HTML/fig14.html')

In [60]:
# convert_json(tinder)

## Statistical Testing 

In [61]:
# filter dataset for only tinder users
tinder_users = tinder.query("answer=='Yes'")

In [62]:
# look at tinder users dataset
tinder_users.head()

,id,usage,university,answer,matches,percentage,relationship
7,293223.0000,university,University of Iowa,Yes,1.000000,0.333000,0
19,292943.0117,university,"University of California, Davis",Yes,8.497078,0.193545,1
30,293163.0000,university,Cal Poly San Luis Obispo,Yes,8.000000,0.320000,0
37,293272.2777,university,"University of California, Santa Cruz",Yes,4.361147,0.323162,1
45,293661.0000,university,Morehead State University,Yes,0.000000,0.000000,0


In [63]:
# count of rows
tinder_users.id.count()

164

In [64]:
# the targets of tinder users
tinder_relat = tinder_users.relationship.to_list()

In [65]:
# dataset of those that dont use tinder
non_users = tinder.query("answer=='No'")

In [66]:
# look at non user dataset
non_users.head()

,id,usage,university,answer,matches,percentage,relationship
0,292890.8970,web,"Meridian, Idaho",No,0.000000,0.000000,0
1,292887.9870,web,"Meridian, Idaho",No,0.000000,0.000000,0
2,292894.0656,gender,"Meridian, Idaho",No,499.173606,0.225255,0
3,292887.1180,web,"Meridian, Idaho",No,0.000000,0.000000,0
4,292893.6561,gender,"Meridian, Idaho",No,455.925963,0.211360,0


In [67]:
# count of rows
non_users.id.count()

1732

In [68]:
# list of non users relationship target
non_relat = non_users.relationship.to_list()

### Proportions Z-test

Null Hypothesis

    * Null Hypothesis that there is no difference in the proportion of successes between Tinder users and non Tinder users datasets

In [69]:
import statsmodels.stats.proportion as smp

# define sample sizes and successes
tinder_use, non_use = 164, 1732
successes1, successes2 = 86, 579

# calculate sample proportions
prop1 = successes1 / tinder_use
prop2 = successes2 / non_use

# conduct the test
z_stat, p_value = smp.proportions_ztest([successes1, successes2], [tinder_use, non_use], alternative='two-sided')

# print the test results
print('Proportions Z-Test:')
print('Z-statistic: {:.2f}'.format(z_stat))
print('P-value: {:.4f}'.format(p_value))
if p_value < 0.05:
    print('Reject null hypothesis: there is a significant difference in proportions.')
else:
    print('Fail to reject null hypothesis: there is no significant difference in proportions.')


Proportions Z-Test:
Z-statistic: 4.88
P-value: 0.0000
Reject null hypothesis: there is a significant difference in proportions.


Statistical testing confirms that the separate datasets are not likely to be the same. Therefore, there seems to be a difference in the relationship rates using tinder, and not using tinder. 

# Looking for Love in All the Wrong Places?

* Are you looking for love? What are your chances of finding it on tinder, and what are your chances of finding it by yourself? 
* Can we predict your chances on tinder based on your school and average stats?


## ML Predicting Relationship

In [70]:
# looking at tinder columns
tinder.columns

Index(['id', 'usage', 'university', 'answer', 'matches', 'percentage',
       'relationship'],
      dtype='object')

In [71]:
# encoding categorical columns
encoder = OrdinalEncoder()
columns_to_encode = ['usage', 'university', 'answer']
data_ordinal = pd.DataFrame(encoder.fit_transform(tinder[columns_to_encode]), columns=columns_to_encode)
tinder.update(data_ordinal)


In [72]:
# reset index after dropping rows
tinder_users.reset_index(inplace=True, drop=True)
non_users.reset_index(inplace=True, drop=True)

In [73]:
# label encoding tinder users dataset
encoder = OrdinalEncoder()
columns_to_encode1 = ['usage', 'university', 'answer']
data_ordinal3 = pd.DataFrame(encoder.fit_transform(tinder_users[columns_to_encode1]), columns=columns_to_encode1)
tinder_users.loc[:, columns_to_encode1] = data_ordinal3

C:\Users\XIX\AppData\Local\Temp\ipykernel_1864\3215945174.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [74]:
# looking at tinder users dataset after encoding
tinder_users.head()

,id,usage,university,answer,matches,percentage,relationship
0,293223.0000,2.0,88.0,0.0,1.000000,0.333000,0
1,292943.0117,2.0,73.0,0.0,8.497078,0.193545,1
2,293163.0000,2.0,9.0,0.0,8.000000,0.320000,0
3,293272.2777,2.0,77.0,0.0,4.361147,0.323162,1
4,293661.0000,2.0,45.0,0.0,0.000000,0.000000,0


In [75]:
# convert_json(tinder_users)

In [76]:
# label encoding tinder users dataset
encoder = OrdinalEncoder()
columns_to_encode = ['usage', 'university', 'answer']
data_ordinal4 = pd.DataFrame(encoder.fit_transform(non_users[columns_to_encode]), columns=columns_to_encode)
non_users.update(data_ordinal4)

C:\Users\XIX\AppData\Local\Temp\ipykernel_1864\1093344357.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [77]:
# look at non users
non_users.head()

,id,usage,university,answer,matches,percentage,relationship
0,292890.8970,3.0,41.0,0.0,0.000000,0.000000,0
1,292887.9870,3.0,41.0,0.0,0.000000,0.000000,0
2,292894.0656,0.0,41.0,0.0,499.173606,0.225255,0
3,292887.1180,3.0,41.0,0.0,0.000000,0.000000,0
4,292893.6561,0.0,41.0,0.0,455.925963,0.211360,0


In [78]:
# convert_json(non_users)

In [79]:
# full dataset features and target
X = tinder.drop(columns=['relationship', 'id'])
y = tinder.relationship

In [80]:
# looking at target balance
y.value_counts()

0    1231
1     665
Name: relationship, dtype: int64

Target values are not balanced

In [81]:
# tinder dataset
X3 = tinder_users.drop(columns=['relationship', 'id'])
y3 = tinder_users.relationship

In [82]:
# target balance of tinder users
y3.value_counts()

1    86
0    78
Name: relationship, dtype: int64

Target values are balanced

In [83]:
# Non Tinder dataset
X4 = non_users.drop(columns=['relationship', 'id'])
y4 = non_users.relationship

In [84]:
# target values
y4.value_counts()

0    1153
1     579
Name: relationship, dtype: int64

Targets are not ballanced

In [85]:
# looking at full dataset features
X.head()

,usage,university,answer,matches,percentage
0,3.0,46.0,0.0,0.000000,0.000000
1,3.0,46.0,0.0,0.000000,0.000000
2,0.0,46.0,0.0,499.173606,0.225255
3,3.0,46.0,0.0,0.000000,0.000000
4,0.0,46.0,0.0,455.925963,0.211360


In [86]:
# convert_json(X)
# convert_json(y)

In [87]:
# looking at tinder users dataset
X3.head()

,usage,university,answer,matches,percentage
0,2.0,88.0,0.0,1.000000,0.333000
1,2.0,73.0,0.0,8.497078,0.193545
2,2.0,9.0,0.0,8.000000,0.320000
3,2.0,77.0,0.0,4.361147,0.323162
4,2.0,45.0,0.0,0.000000,0.000000


In [88]:
# convert_json(X3)

In [89]:
# looking at non user dataset
X4.head()

,usage,university,answer,matches,percentage
0,3.0,41.0,0.0,0.000000,0.000000
1,3.0,41.0,0.0,0.000000,0.000000
2,0.0,41.0,0.0,499.173606,0.225255
3,3.0,41.0,0.0,0.000000,0.000000
4,0.0,41.0,0.0,455.925963,0.211360


In [90]:
# convert_json(X4)

#### Usage Map

In [91]:
# What is your medium of usage?
usage = pd.DataFrame({'usage':['Gender', 'Mobile', 'University', 'Web'], 'values': [0, 1 , 2 ,3]})
usage

,usage,values
0,Gender,0
1,Mobile,1
2,University,2
3,Web,3


In [92]:
# convert_json(usage)

In [93]:
# show  map of coded values
tinder.usage.value_counts()

0.0    488
1.0    483
2.0    464
3.0    461
Name: usage, dtype: int64

In [94]:
# label encoded values
X.usage.value_counts()

0.0    488
1.0    483
2.0    464
3.0    461
Name: usage, dtype: int64

#### University Map

In [95]:
# What University do you attend?
uni = pd.DataFrame({'university':['Meridian, Idaho', 'Westport, CT', 'University of Pennsylvania', 'University of Mississippi', 'University of Washington', 'University of Southern California', 'Washington State University', 'University of Tampa', 'Wesleyan University', 'University of Wisconsin, Eau Claire', 'University of California, Santa Cruz', 'University of California, Berkeley', 'Williams College', 'Vassar College', 'Vanderbilt University', 'University of Buffalo', 'University of Oregon', 'Union College', 'Whatsgoodly University', 'University of Delaware']
, 'values': [46.0, 144.0, 123.0, 117.0, 133.0, 127.0, 140.0, 129.0, 143.0, 134.0, 97.0, 92.0, 146.0, 139.0, 138.0, 91.0, 122.0, 86.0, 145.0, 104.0]})
uni

,university,values
0,"Meridian, Idaho",46.0
1,"Westport, CT",144.0
2,University of Pennsylvania,123.0
3,University of Mississippi,117.0
4,University of Washington,133.0
5,University of Southern California,127.0
6,Washington State University,140.0
7,University of Tampa,129.0
8,Wesleyan University,143.0
9,"University of Wisconsin, Eau Claire",134.0


In [96]:
# convert_json(uni)

In [97]:
# show  map of coded values
tinder.university.value_counts().nlargest(20)

46.0     1421
144.0      14
123.0      12
117.0      10
133.0      10
127.0       9
140.0       8
129.0       8
143.0       8
134.0       8
97.0        8
92.0        8
146.0       7
139.0       7
138.0       7
91.0        7
122.0       7
86.0        7
145.0       6
104.0       6
Name: university, dtype: int64

In [98]:
# coded values
X.university.value_counts().nlargest(20)

46.0     1421
144.0      14
123.0      12
117.0      10
133.0      10
127.0       9
140.0       8
129.0       8
143.0       8
134.0       8
97.0        8
92.0        8
146.0       7
139.0       7
138.0       7
91.0        7
122.0       7
86.0        7
145.0       6
104.0       6
Name: university, dtype: int64

May be easier to assume Meridian, as it is the most prevalent, or at least one of the top 20 universities.

##### Answer Map

In [99]:
# Do you use Tinder?
answer = pd.DataFrame({'answer':['No', 'Yes'], 'values': [0, 1 ]})
answer

,answer,values
0,No,0
1,Yes,1


In [100]:
# convert_json(answer)

In [101]:
# map of coded values
tinder.answer.value_counts()

0.0    1732
1.0     164
Name: answer, dtype: int64

In [102]:
# coded values
X.answer.value_counts()

0.0    1732
1.0     164
Name: answer, dtype: int64

In [103]:
# new data to make predictions
new_data = pd.DataFrame(index=['usage', 'university', 'answer', 'matches', 'percentage'], data=[2, 46, 0, 400, 0.25])
new_data = new_data.T

In [104]:
# looking at new testing dataset
new_data

,usage,university,answer,matches,percentage
0,2.0,46.0,0.0,400.0,0.25


In [105]:
# convert_json(new_data)

In [111]:
# Classifier pipeline for Full dataset
pipe_lr = make_pipeline(StandardScaler(), SMOTE(random_state=19), LogisticRegression(random_state=19, max_iter=1000))
pipe_dt = make_pipeline(StandardScaler(), SMOTE(random_state=19), DecisionTreeClassifier(random_state=19))
pipe_rf = make_pipeline(StandardScaler(), SMOTE(random_state=19), RandomForestClassifier(random_state=19))
pipe_sv = make_pipeline(StandardScaler(), SMOTE(random_state=19), svm.SVC(random_state=19))
pipe_nb = make_pipeline(StandardScaler(), SMOTE(random_state=19), GaussianNB())
pipe_kn = make_pipeline(StandardScaler(), SMOTE(random_state=19), KNeighborsClassifier(n_neighbors=3))
pipe_xg = make_pipeline(StandardScaler(), SMOTE(random_state=19), XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=100, random_state=19))
pipe_lb = make_pipeline(StandardScaler(), SMOTE(random_state=19), lightgbm.LGBMClassifier(objective='binary', metric='f1', boosting_type='gbdt', random_state=19))

pipelines = [pipe_lr, pipe_dt, pipe_rf, pipe_sv, pipe_nb, pipe_kn, pipe_xg, pipe_lb]

best_accuracy = 0
best_classifier = 0
best_pipeline = ""

pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'Random Forest', 3: 'SVM', 4: 'Naive-Bayes', 5: 'KNN', 6:'XG Boost', 7:'LGBM'}

# Use cross-validation to evaluate the models
for i, model in enumerate(pipelines):
    scores = cross_val_score(model, X, y, cv=5, scoring='f1')
    print('{} Cross-Validation Accuracy: {:.2f}'.format(pipe_dict[i], scores.mean()))
    if scores.mean() > best_accuracy:
        best_accuracy = scores.mean()
        best_pipeline = model
        best_classifier = i

# Print the best classifier
print('\nClassifier with the best F1 Score: {}'.format(pipe_dict[best_classifier]))


Logistic Regression Cross-Validation Accuracy: 0.71
Decision Tree Cross-Validation Accuracy: 0.85
Random Forest Cross-Validation Accuracy: 0.86
SVM Cross-Validation Accuracy: 0.81
Naive-Bayes Cross-Validation Accuracy: 0.68
KNN Cross-Validation Accuracy: 0.86
XG Boost Cross-Validation Accuracy: 0.86
LGBM Cross-Validation Accuracy: 0.87

Classifier with the best F1 Score: LGBM


In [112]:
# Classifier pipeline for Tinder Dataset
pipe_lr2 = make_pipeline(StandardScaler(), SMOTE(random_state=19), LogisticRegression(random_state=19, max_iter=1000))
pipe_dt2 = make_pipeline(StandardScaler(), SMOTE(random_state=19), DecisionTreeClassifier(random_state=19))
pipe_rf2 = make_pipeline(StandardScaler(), SMOTE(random_state=19), RandomForestClassifier(random_state=19))
pipe_sv2 = make_pipeline(StandardScaler(), SMOTE(random_state=19), svm.SVC(random_state=19))
pipe_nb2 = make_pipeline(StandardScaler(), SMOTE(random_state=19), GaussianNB())
pipe_kn2 = make_pipeline(StandardScaler(), SMOTE(random_state=19), KNeighborsClassifier(n_neighbors=3))
pipe_xg2 = make_pipeline(StandardScaler(), SMOTE(random_state=19), XGBClassifier(random_state=19, max_depth=2, learning_rate=0.2, n_estimators=50))
pipe_lb2 = make_pipeline(StandardScaler(), SMOTE(random_state=19), lightgbm.LGBMClassifier(objective='binary', metric='f1', boosting_type='gbdt', random_state=19))

pipelines = [pipe_lr2, pipe_dt2, pipe_rf2, pipe_sv2, pipe_nb2, pipe_kn2, pipe_xg2, pipe_lb2]

best_accuracy = 0
best_classifier = 0
best_pipeline = ""

pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'Random Forest', 3: 'SVM', 4: 'Naive-Bayes', 5: 'KNN', 6:'XG Boost', 7:'LGBM'}
# Use cross-validation to evaluate the models
for i, model in enumerate(pipelines):
    scores = cross_val_score(model, X3, y3, cv=5, scoring='f1')
    print('{} Cross-Validation Accuracy: {:.2f}'.format(pipe_dict[i], scores.mean()))
    if scores.mean() > best_accuracy:
        best_accuracy = scores.mean()
        best_pipeline = model
        best_classifier = i

# Print the best classifier
print('\nClassifier with the best F1 Score: {}'.format(pipe_dict[best_classifier]))



Logistic Regression Cross-Validation Accuracy: 0.63
Decision Tree Cross-Validation Accuracy: 0.72
Random Forest Cross-Validation Accuracy: 0.68
SVM Cross-Validation Accuracy: 0.67
Naive-Bayes Cross-Validation Accuracy: 0.28
KNN Cross-Validation Accuracy: 0.67
XG Boost Cross-Validation Accuracy: 0.73
LGBM Cross-Validation Accuracy: 0.66

Classifier with the best F1 Score: XG Boost


In [113]:
# Classifier pipeline for non user Dataset
pipe_lr3 = make_pipeline(StandardScaler(), SMOTE(random_state=19), LogisticRegression(random_state=19, max_iter=1000))
pipe_dt3 = make_pipeline(StandardScaler(), SMOTE(random_state=19), DecisionTreeClassifier(random_state=19))
pipe_rf3 = make_pipeline(StandardScaler(), SMOTE(random_state=19), RandomForestClassifier(random_state=19))
pipe_sv3 = make_pipeline(StandardScaler(), SMOTE(random_state=19), svm.SVC(random_state=19))
pipe_nb3 = make_pipeline(StandardScaler(), SMOTE(random_state=19), GaussianNB())
pipe_kn3 = make_pipeline(StandardScaler(), SMOTE(random_state=19), KNeighborsClassifier(n_neighbors=3))
pipe_xg3 = make_pipeline(StandardScaler(), SMOTE(random_state=19), XGBClassifier(random_state=19, max_depth=3, learning_rate=0.1, n_estimators=100))
pipe_lb3 = make_pipeline(StandardScaler(), SMOTE(random_state=19), lightgbm.LGBMClassifier(objective='binary', metric='f1', boosting_type='gbdt', random_state=19))

pipelines = [pipe_lr3, pipe_dt3, pipe_rf3, pipe_sv3, pipe_nb3, pipe_kn3, pipe_xg3, pipe_lb3]

best_accuracy = 0
best_classifier = 0
best_pipeline = ""

pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'Random Forest', 3: 'SVM', 4: 'Naive-Bayes', 5: 'KNN', 6:'XG Boost', 7:'LGBM'}

# Use cross-validation to evaluate the models
for i, model in enumerate(pipelines):
    scores = cross_val_score(model, X4, y4, cv=5, scoring='f1')
    print('{} Cross-Validation Accuracy: {:.2f}'.format(pipe_dict[i], scores.mean()))
    if scores.mean() > best_accuracy:
        best_accuracy = scores.mean()
        best_pipeline = model
        best_classifier = i

# Print the best classifier
print('\nClassifier with the best F1 Score: {}'.format(pipe_dict[best_classifier]))


Logistic Regression Cross-Validation Accuracy: 0.73
Decision Tree Cross-Validation Accuracy: 0.88
Random Forest Cross-Validation Accuracy: 0.88
SVM Cross-Validation Accuracy: 0.84
Naive-Bayes Cross-Validation Accuracy: 0.74
KNN Cross-Validation Accuracy: 0.88
XG Boost Cross-Validation Accuracy: 0.89
LGBM Cross-Validation Accuracy: 0.89

Classifier with the best F1 Score: LGBM


##### Why evaluate on f1 score

The F1 score is a commonly used evaluation metric in machine learning that combines precision and recall. When working with imbalanced datasets, where the number of samples in each class is unequal, accuracy may not be a reliable metric to evaluate the performance of a model. For example, if the majority of the samples belong to the negative class, a model that always predicts the negative class will have high accuracy, even though it is not predicting the positive class correctly.

In such scenarios, the F1 score can be a better metric to use, as it takes into account both precision and recall, which are important factors in assessing a model's performance in such situations. The F1 score balances precision and recall and gives a single metric that summarizes the performance of a binary classification model.

Moreover, in situations where the dataset is imbalanced, the F1 score is less sensitive to false negatives than other metrics such as precision or recall. In other words, the F1 score is less likely to be biased towards the majority class, and it provides a more accurate representation of the model's performance on both classes.

#### Full Dataset Final Model

In [114]:
# split into train and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=19)

In [115]:
# Fit data on training set
pipe_lb.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('smote', SMOTE(random_state=19)),
                ('lgbmclassifier',
                 LGBMClassifier(metric='f1', objective='binary',
                                random_state=19))])

In [116]:
# make predictions on test set
pred = pipe_lb.predict(X_test)

In [117]:
# f1 score on test set
f1_score(y_test, pred)

0.8929889298892989

In [118]:
# classification report 
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94       250
           1       0.86      0.93      0.89       130

    accuracy                           0.92       380
   macro avg       0.91      0.93      0.92       380
weighted avg       0.93      0.92      0.92       380



The classification report shows high accuracy and precision with the negative class, with an overall high F1 score. The positive class has good precision, and very good recall, with the F1 score being fairly high. 

In [119]:
# confusion matrix
confusion_matrix(y_test, pred)

array([[230,  20],
       [  9, 121]], dtype=int64)

In [120]:
# Full dataset confusion matrix
px.imshow(confusion_matrix(y_test, pred), text_auto=True, labels=dict(x="Actual", y="Predictions", color="Count"),
                x=['Relationship', 'Single'],
                y=['Relationship', 'Single'])

The confusion matrix illustrates 230 True Positives, 20 False Positives, 121 True Negatives, and 9 False Negatives. 

In [121]:
# make predictions of new data
pipe_lb.predict_proba(new_data)

array([[0.06000515, 0.93999485]])

In [122]:
# class result of prediction
pipe_lb.predict(new_data)

array([1], dtype=int64)

Predicting target of a relationship with a 94% probability, based on the model trained on the full dataset, and the new data. The F1 score on the test set is good, so these results have a good amount of credibility. 

#### Tinder Users Final Model

In [123]:
# look at dataset
X3.head()

,usage,university,answer,matches,percentage
0,2.0,88.0,0.0,1.000000,0.333000
1,2.0,73.0,0.0,8.497078,0.193545
2,2.0,9.0,0.0,8.000000,0.320000
3,2.0,77.0,0.0,4.361147,0.323162
4,2.0,45.0,0.0,0.000000,0.000000


In [124]:
# split into train and test set
X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3, y3, test_size=0.20, random_state=19)

In [125]:
# Fit data on training set
pipe_xg2.fit(X3_train, y3_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('smote', SMOTE(random_state=19)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=0, gpu_id=-1,
                               grow_policy='depthwise', importance_type=None,
                               interaction_constraints='', learning_rate=0.2,
                               max_bin=256, max_cat_threshold=64,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=2, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimator

In [126]:
# make predictions on test set
pred2 = pipe_xg2.predict(X3_test)

In [127]:
# f1 score on test set
f1_score(y3_test, pred2)

0.5714285714285715

In [128]:
# classification report 
print(classification_report(y3_test, pred2))

              precision    recall  f1-score   support

           0       0.62      0.44      0.52        18
           1       0.50      0.67      0.57        15

    accuracy                           0.55        33
   macro avg       0.56      0.56      0.54        33
weighted avg       0.56      0.55      0.54        33



The classification report shows mediocre accuracy and precision with the negative class, with an overall mediocre F1 score. The positive class also has mediocre precision and recall, with the F1 score being mediocre as well. This is likely due to the small size of the dataset, and subsequently, the test set. We would need a larger subset of tinder users to create a better model. 

In [129]:
# confusion matrix
confusion_matrix(y3_test, pred2)

array([[ 8, 10],
       [ 5, 10]], dtype=int64)

In [130]:
# Tinder dataset confusion matrix
px.imshow(confusion_matrix(y3_test, pred2), text_auto=True, labels=dict(x="Actual", y="Predictions", color="Count"),
                x=['Relationship', 'Single'],
                y=['Relationship', 'Single'])

The confusion matrix illustrates 8 True Positives, 10 False Positives, 10 True Negatives, and 5 False Negatives. This explains why the F1 score is mediocre, as there are more False Positives than True Positives.

In [131]:
# make predictions of new data
pipe_xg2.predict_proba(new_data)

array([[0.9685164 , 0.03148358]], dtype=float32)

In [132]:
# class result of prediction
pipe_xg2.predict(new_data)

array([0])

Predicting target of not a relationship with a 96.8% probability, based on the model trained on the tinder dataset, and the new data. The overall F1 score of the model is average, so the results are not expected to be the best. 

#### Non Tinder Final Model

In [133]:
# look at dataset
X4.head()

,usage,university,answer,matches,percentage
0,3.0,41.0,0.0,0.000000,0.000000
1,3.0,41.0,0.0,0.000000,0.000000
2,0.0,41.0,0.0,499.173606,0.225255
3,3.0,41.0,0.0,0.000000,0.000000
4,0.0,41.0,0.0,455.925963,0.211360


In [134]:
# split into train and test set
X4_train, X4_test, y4_train, y4_test = train_test_split(
    X4, y4, test_size=0.20, random_state=19)

In [135]:
# Fit data on training set
pipe_lb3.fit(X4_train, y4_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('smote', SMOTE(random_state=19)),
                ('lgbmclassifier',
                 LGBMClassifier(metric='f1', objective='binary',
                                random_state=19))])

In [136]:
# make predictions on test set
pred3 = pipe_lb3.predict(X4_test)

In [137]:
# f1 score on test set
f1_score(y4_test, pred3)

0.8861788617886179

In [138]:
# classification report 
print(classification_report(y4_test, pred3))

              precision    recall  f1-score   support

           0       0.95      0.93      0.94       227
           1       0.87      0.91      0.89       120

    accuracy                           0.92       347
   macro avg       0.91      0.92      0.91       347
weighted avg       0.92      0.92      0.92       347



The classification report shows high accuracy and precision with the negative class, with an overall high F1 score. The positive class also has high precision and recall, with the F1 score being high as well. 

In [139]:
# confusion matrix
confusion_matrix(y4_test, pred3)

array([[210,  17],
       [ 11, 109]], dtype=int64)

In [140]:
# non users dataset confusion matrix
px.imshow(confusion_matrix(y4_test, pred3), text_auto=True, labels=dict(x="Actual", y="Predictions", color="Count"),
                x=['Relationship', 'Single'],
                y=['Relationship', 'Single'])

The confusion matrix illustrates 210 True Positives, 17 False Positives, 109 True Negatives, and 11 False Negatives. This explains why the F1 score is high, as there are more correct predictions of the correct class.

In [141]:
# make predictions of new data
pipe_lb3.predict_proba(new_data)

array([[0.03448091, 0.96551909]])

In [142]:
# class result of prediction
pipe_lb3.predict(new_data)

array([1], dtype=int64)

Predicting target of a relationship with a 96.6% probability, based on the model trained on the non tinder user dataset, and the new data. The overall F1 score of the model is good, so the results have good credibility. 

## Save model to JSON

# Save model as json 
tinder_users_json = xgbc5.to_json()
with open("model.json", "w") as json_file:
    json_file.write(tinder_users_json)

In [143]:
# Save the pipeline as a binary file
# joblib.dump(pipe_lb, 'full_model.joblib')

In [144]:
# Save the pipeline as a binary file
# joblib.dump(pipe_xg2, 'tinder_users_model.joblib')


In [145]:
# Save the pipeline as a binary file
# joblib.dump(pipe_lb3, 'non_users_model.joblib')

In [146]:
# Load the pipeline from the binary file
full_model = joblib.load('ML Models/full_model.joblib')
tinder_user_model = joblib.load('ML Models/tinder_users_model.joblib')
non_user_model = joblib.load('ML Models/non_users_model.joblib')

## Conclusion

We conducted EDA on the tinder dataset and highlighted some trends. We found more relationships with those that used tinder, compared to those that did not use tinder. This occurrence was confirmed with hypothesis testing, which suggested the relationship occurrences were different among tinder users and non users. 

We then created a machine learning model to predict the relationship status of an individual, based on a few self reported features, and two average features set by values extracted from the dataset. Initially, the dataset was split into three groups: the full dataset, tinder users, and non users. The full dataset model had the best F1 score, while the non users model came a close second. A good F1 score suggests the model has good precision and recall, in other words, good at predicting both positive and negative classes. The tinder users model had an average F1 score, so the model does not accurately predict both the positive and negative classes. This was due to the small nature of the tinder dataset, which also left very little data to test the model. 